## Support Vector Machines - Bag of Words model

In [ ]:
# Import the necessary libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
data = pd.read_csv('balanced.csv')

In [ ]:
# Turn label into discrete variable
label_encoder = LabelEncoder()
data['condition_label'] = label_encoder.fit_transform(data['condition_label'])

In [ ]:
# Split data into training and testing sets
X = data['clean_text']
y = data['condition_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [ ]:
# Convert text data into numerical features with TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

In [ ]:
# Define the hyperparameter grid for SVM
param_grid = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
}

In [ ]:
# Use GridSearchCV to define best hyperparameters
svm_classifier = SVC()

grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train_counts, y_train)

best_params = grid_search.best_params_
best_svm_classifier = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)

In [ ]:
# Make predictions on the test data
y_pred = best_svm_classifier.predict(X_test_counts)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with best hyperparameters: {accuracy}")

In [ ]:
# Print classification report
print(classification_report(y_test, y_pred))

In [ ]:
# Define and display confusion matrix
def plot_confusion_matrix(y_test, y_pred, labels):
    cm = confusion_matrix(y_test, y_pred, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    fig, ax = plt.subplots(figsize=(6,6))
    plt.rcParams.update({'font.size':13, 'font.monospace':'Computer Modern Typewriter'})
    disp.plot(ax=ax, cmap = "binary", colorbar=False)

plot_confusion_matrix(y_test, y_pred,   labels=best_svm_classifier.classes_)